# 04. Data Processing

### Importando as bibliotecas

In [1]:
import sys
sys.path.append('..')
import pandas as pd
from ydata_profiling import ProfileReport
import functions.fn_stats as fn_stats
import functions.fn_charts as fn_charts
import params.consts as consts

### Lendo o dataset

In [2]:
df = pd.read_csv(consts.DATASET_RAW) # Armazenando o dataset em uma variável

In [3]:
df.head(5) # Exibindo as 3 primeiras linhas do dataset

,Tipo,Data,Curtidas,Comentários,Visualizações,Tags,Pessoas,Campanhas,Carrossel,Interacoes
0,Foto,11-Sep,"2,858",16,NaN,Loja,N,N,NaN,"2,874"
1,Foto,11-Sep,"2,930",28,NaN,Loja/Produtos,N,N,NaN,"2,958"
2,Foto,11-Sep,"2,807",9,NaN,Loja,N,N,NaN,"2,816"
3,Vídeo,12-Sep,"5,115",49,"82,878",Produtos,N,N,NaN,"5,164"
4,Foto,13-Sep,"4,392",45,NaN,Produtos,S,N,NaN,"4,437"


In [4]:
df.info() # Exibindo as informações das variáveis do dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Tipo           52 non-null     object
 1   Data           52 non-null     object
 2   Curtidas       52 non-null     object
 3   Comentários    52 non-null     int64 
 4   Visualizações  15 non-null     object
 5   Tags           44 non-null     object
 6   Pessoas        52 non-null     object
 7   Campanhas      52 non-null     object
 8   Carrossel      8 non-null      object
 9   Interacoes     52 non-null     object
dtypes: int64(1), object(9)
memory usage: 4.2+ KB


In [5]:
fn_stats.describe(df) # Usando a função que exibe as estatísticas das colunas numéricas

,Comentários
count,52.00
mean,189.50
std,170.69
min,9.00
25%,69.50
50%,128.00
75%,265.25
max,852.00


In [6]:
df.describe(exclude='number') # Exibindo as estatísticas das colunas categóricas

,Tipo,Data,Curtidas,Visualizações,Tags,Pessoas,Campanhas,Carrossel,Interacoes
count,52,52,52,15,44,52,52,8,52
unique,4,50,52,15,12,2,2,1,52
top,Foto,11-Sep,"2,858","82,878",Produtos,S,N,S,"2,874"
freq,36,3,1,1,12,40,30,8,1


In [7]:
df.isnull().sum() # Somando todos os valores nulos de cada coluna

Tipo              0
Data              0
Curtidas          0
Comentários       0
Visualizações    37
Tags              8
Pessoas           0
Campanhas         0
Carrossel        44
Interacoes        0
dtype: int64

### Corrigindo os tipos de dados das colunas
- **Data:** Convertendo essa coluna para o formato de data.
- **Curtidas:** Convertendo essa coluna para o formato de float.
- **Interacoes:** Convertendo essa coluna para o formato de float.

In [8]:
df['Data'] = pd.to_datetime(df['Data'] + '-2021', format='%d-%b-%Y') # Convertendo a coluna para o formato de data
df.rename(columns={'Interacoes ': 'Interacoes'}, inplace=True) # Removendo o espaço do nome da coluna
for column in ['Curtidas', 'Interacoes']: # Criando uma estrutura de repetição para tratar as colunas Curtidas e Interacoes
    df[column] = df[column].str.replace(',', '.', regex=False) # Substituindo vírgulas por pontos
    df[column] = df[column].astype(float) # Convertendo a coluna para o formato float
    df[column] = df[column] * 1000 # Definindo o valor correto para coluna
    df[column] = df[column].astype('int64') # Convertendo a coluna para o formato int

### Tratando as colunas com valores nulos/vazios
- **Visualizações:** Excluindo essa coluna pois há muitos valores vazios.
- **Tags:** Substituindo os valores vazios por Sem Tag, após analisar a base.
- **Carrossel:** Substituindo os valores vazios por N, após analisar a base.

In [9]:
df = df.drop('Visualizações', axis=1) # Excluindo a coluna
df['Tags'] = df['Tags'].fillna('Sem Tag') # Substituindo os valores vazios
df['Carrossel'] = df['Carrossel'].fillna('N') # Substituindo os valores vazios

### Criando colunas de tempo/idade
- **N/A**

### Mesclando colunas com muitas categorias
- **N/A**

### Unificando colunas semelhantes
- **N/A**

### Criando colunas derivadas de outras
- **Dia_Da_Semana:** Criando a coluna de dia da semana com base na data para extrair possíveis insights

In [10]:
df['Dia_Da_Semana'] = df['Data'].dt.day_name() # Criando a coluna de dia da semana com base na data

### Visualizando o dataset antes de tratar outliers e excluir colunas

In [11]:
df.head(5) # Exibindo as 5 primeiras linhas do dataset

,Tipo,Data,Curtidas,Comentários,Tags,Pessoas,Campanhas,Carrossel,Interacoes,Dia_Da_Semana
0,Foto,2021-09-11,2858,16,Loja,N,N,N,2874,Saturday
1,Foto,2021-09-11,2930,28,Loja/Produtos,N,N,N,2958,Saturday
2,Foto,2021-09-11,2807,9,Loja,N,N,N,2816,Saturday
3,Vídeo,2021-09-12,5115,49,Produtos,N,N,N,5164,Sunday
4,Foto,2021-09-13,4392,45,Produtos,S,N,N,4437,Monday


### Tratando as colunas com valores outliers
- **N/A**

### Excluindo as colunas que contém valores únicos
- **N/A**

### Excluindo as colunas que contém valores constantes
- **N/A**

### Salvando o dataset processado
- **N/A**

### Excluindo as colunas auxiliares
- **N/A**

### Ordenando as colunas relacionadas

In [12]:
df = df.reindex(columns=[ # Ordenando as colunas em uma ordem que faz mais sentido para análises
        'Tipo',
        'Data',
        'Dia_Da_Semana',
        'Curtidas',
        'Comentários',
        'Interacoes',
        'Tags',
        'Pessoas',
        'Campanhas',
        'Carrossel',
    ]
)

### Salvando o dataset lean

In [13]:
df.to_csv(consts.DATASET_LEAN, index=False) # Salvando o dataset lean

### Gerando um novo relatório Profile Report

In [14]:
profile = ProfileReport(df) # Armazenando em uma variável o ProfileReport

profile.to_file(consts.EDA_1) # Salvando o profile report na pasta reports em formato html

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]